In [16]:
import numpy as np
import pandas as pd
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.INFO)

In [17]:

filenames = tf.train.match_filenames_once("./train/*.jpg")
fileq = tf.train.string_input_producer(filenames)


In [18]:
def _preprocess(image) :
    image = tf.image.random_flip_left_right(image)
    #image = crop(image)
    image = tf.transpose(image,[2,0,1]) #change to CHW format
    image = (tf.cast(image,tf.float32) - 128.0)/128.0 #push in to [-1 to 1] area.
    return image

In [19]:
image_reader = tf.WholeFileReader()
(label,image_file) = image_reader.read(fileq)
image = tf.image.decode_jpeg(image_file,3)
resized = tf.image.resize_images(image,[180,180],tf.image.ResizeMethod.BILINEAR)
preprocessed = _preprocess(resized)



In [20]:
input_ = tf.placeholder(tf.float32,shape =[None,180,180,3], name='input_')
target_= tf.placeholder(tf.float32,shape =[None,120], name='target_')
keep_prob= tf.placeholder(tf.float32, name='keep_prob')


In [21]:
filters =32
kernel_size=(5,5)
strides=(2,2)

initializer=tf.contrib.layers.xavier_initializer_conv2d()

conv1 = tf.layers.conv2d(input_,filters,kernel_size,strides,padding='same',kernel_initializer=initializer)
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)


filters =64
kernel_size=(5,5)
strides=(2,2)
conv2 = tf.layers.conv2d(pool1,filters,kernel_size,strides,padding='same',kernel_initializer=initializer)
pool2 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)



In [22]:
pool2_flat = tf.contrib.layers.flatten(pool2)

num_units=512
dense1 = tf.layers.dense(inputs=pool2_flat, units=num_units, activation=tf.nn.relu)
dropout1 = tf.layers.dropout(inputs=dense1, rate=keep_prob)


dense2 = tf.layers.dense(inputs=dropout1, units=256, activation=tf.nn.relu)
dropout2 = tf.layers.dropout(inputs=dense2, rate=keep_prob)

logits = tf.layers.dense(inputs=dropout2, units=120)


In [23]:
# Calculate Loss (for both TRAIN and EVAL modes)
logits = tf.identity(logits, name='logits')
loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=target_ )

cost = tf.reduce_mean(loss)
optimizer= tf.train.AdamOptimizer().minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(target_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')


In [24]:
def print_stats(session, image_batch, label_batch, cost, accuracy):

    print('Cost: {:f}'.format(session.run(cost,feed_dict={ input_: image_batch,target_: label_batch,keep_prob:1})))
    print('Accuracy: {:f}'.format(session.run(accuracy,feed_dict={ input_: feature_batch,target_: label_batch,keep_prob:1})))

In [27]:

with tf.Session() as sess:
   # initialize the variables
    sess.run((tf.global_variables_initializer(), tf.local_variables_initializer()))
  
  # initialize the queue threads to start to shovel data
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    image,label = tf.train.batch(
                                    [preprocessed,label],
                                    batch_size=5
                                    #,num_threads=1
                                    )
    print(image, label)
    print("from the train set:")
    for i in range(5):
        print(image)

Tensor("batch_7:0", shape=(5, 3, 180, 180), dtype=float32) Tensor("batch_7:1", shape=(5, 5), dtype=string)
from the train set:
Tensor("batch_7:0", shape=(5, 3, 180, 180), dtype=float32)
Tensor("batch_7:0", shape=(5, 3, 180, 180), dtype=float32)
Tensor("batch_7:0", shape=(5, 3, 180, 180), dtype=float32)
Tensor("batch_7:0", shape=(5, 3, 180, 180), dtype=float32)
Tensor("batch_7:0", shape=(5, 3, 180, 180), dtype=float32)
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: input_producer/input_producer_EnqueueMany = QueueEnqueueManyV2[Tcomponents=[DT_STRING], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](input_producer, input_producer/RandomShuffle)]]


## 